***********************************************
## Updated Score : 81.101 %
https://www.kaggle.com/pranjalverma08/tps-april-21-ann-pseudo-label-score-81-101
**********************************************
### Score of this Notebook : 80.782 %

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import theano
import tensorflow
import keras
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import plot_model
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from sklearn.experimental import enable_halving_search_cv  
from sklearn.model_selection import HalvingGridSearchCV
from sklearn.model_selection import HalvingRandomSearchCV

In [ ]:
train=pd.read_csv('../input/tabular-playground-series-apr-2021/train.csv')
test=pd.read_csv('../input/tabular-playground-series-apr-2021/test.csv')

In [ ]:
print(train.shape)
print(test.shape)

# Feature Engineering

In [ ]:
train['Survived'].value_counts()
train.isnull().sum()
test.isnull().sum()
#train.head()
train['Ticket'].nunique()
train[["SibSp", "Survived"]].groupby(['SibSp'], as_index=False).mean().sort_values(by='Survived', ascending=False)
train[["Sex", "Survived"]].groupby(['Sex'], as_index=False).mean().sort_values(by='Survived', ascending=False)
train[["Parch", "Survived"]].groupby(['Parch'], as_index=False).mean().sort_values(by='Survived', ascending=False)
train[["Embarked", "Survived"]].groupby(['Embarked'], as_index=False).mean().sort_values(by='Survived', ascending=False)
train_copy = train.copy()
test_copy = test.copy()
train = train.drop(['PassengerId', 'Ticket'], axis = 1)
test = test.drop(['PassengerId', 'Ticket',], axis = 1)
combine = [train, test]

train['Cabin'].fillna('U', inplace=True)
train['Cabin'] = train['Cabin'].apply(lambda x: x[0])

test['Cabin'].fillna('U', inplace=True)
test['Cabin'] = test['Cabin'].apply(lambda x: x[0])

train['Cabin'].unique()
for dataset in combine:
  dataset['Cabin'] = dataset['Cabin'].fillna('U')
  dataset['Cabin'] = dataset['Cabin'].apply(lambda x: x[0])
  
pd.crosstab(train['Cabin'], train['Survived'])
train[['Cabin', 'Survived']].groupby(['Cabin'], as_index = False).mean().sort_values(by = 'Survived', ascending = True)
cabin_mapping = {"T": 0, "U": 1, "A": 2, "G": 3, "C": 4, "F": 5, "B": 6, "E": 7, "D": 8}
for dataset in combine:
    dataset['Cabin'] = dataset['Cabin'].map(cabin_mapping)
    dataset['Cabin'] = dataset['Cabin'].fillna(0)

#train.head()
for dataset in combine:
    dataset['Title'] = dataset['Name'].map(lambda x: x.split(',')[1].split('.')[0].strip())

pd.crosstab(train['Title'], train['Sex'])


# for dataset in combine:
#     dataset['Title'] = dataset['Title'].replace(['Lady', 'Countess','Capt', 'Col','Don', 'Major', 'Rev', 'Jonkheer', 'Dona'], 'Rare')

#     dataset['Title'] = dataset['Title'].replace('Mlle', 'Miss')
#     dataset['Title'] = dataset['Title'].replace('Ms', 'Miss')
#     dataset['Title'] = dataset['Title'].replace('Mme', 'Mrs')
#     dataset['Title'] = dataset['Title'].replace('Sir', 'Mr')
#     dataset['Title'] = dataset['Title'].replace('Dr', 'Mr')
    
# train[['Title', 'Survived']].groupby(['Title'], as_index=False).mean()

# title_mapping = {"Mrs": 4, "Miss": 3, "Mr": 1, "Master": 2, "Rare": 0}
# for dataset in combine:
#     dataset['Title'] = dataset['Title'].map(title_mapping)
#     dataset['Title'] = dataset['Title'].fillna(0)

#train.head()

for dataset in combine:
    dataset['Sex'] = dataset['Sex'].map( {'female': 1, 'male': 0} ).astype(int)

#train.head()
# Imputing Missing Values

train['Age'].fillna(train['Age'].dropna().median(), inplace=True)
test['Age'].fillna(train['Age'].mean(), inplace = True)
test['Fare'].fillna(train['Fare'].dropna().median(), inplace = True)
train['Embarked'].fillna('C', inplace = True)
test['Embarked'].fillna('C', inplace = True)
train['Fare'].fillna(train['Fare'].dropna().median(), inplace = True)
train.drop('Title',axis=1,inplace=True)
test.drop('Title',axis=1,inplace=True)

train.isnull().sum()
test.isnull().sum()

train['AgeBand'] = pd.cut(train['Age'], 5)
train[['AgeBand', 'Survived']].groupby(['AgeBand'], as_index=False).mean().sort_values(by='AgeBand', ascending=True)


for dataset in combine:    
    dataset.loc[ dataset['Age'] <= 16, 'Age'] = 4
    dataset.loc[(dataset['Age'] > 16) & (dataset['Age'] <= 32), 'Age'] = 1
    dataset.loc[(dataset['Age'] > 32) & (dataset['Age'] <= 48), 'Age'] = 2
    dataset.loc[(dataset['Age'] > 48) & (dataset['Age'] <= 64), 'Age'] = 3
    dataset.loc[ dataset['Age'] > 64, 'Age'] = 0
    
#train.head()



for dataset in combine:
    dataset['FamilySize'] = dataset['SibSp'] + dataset['Parch'] + 1

train[['FamilySize', 'Survived']].groupby(['FamilySize'], as_index=False).mean().sort_values(by='Survived', ascending=False)
'''
for dataset in combine:    
    dataset.loc[ dataset['FamilySize'] > 4, 'FamilySize'] = 0  
    dataset.loc[ dataset['FamilySize'] <= 4, 'FamilySize'] = 1
    
train.head()

'''
for dataset in combine:
    dataset['Embarked'] = dataset['Embarked'].map( {'S': 0, 'C': 1, 'Q': 2} ).astype(int)

#train.head()

train['FareBand'] = pd.qcut(train['Fare'], 4)
train[['FareBand', 'Survived']].groupby(['FareBand'], as_index=False).mean().sort_values(by='FareBand', ascending=True)



for dataset in combine:
    dataset.loc[ dataset['Fare'] <= 7.91, 'Fare'] = 0
    dataset.loc[(dataset['Fare'] > 7.91) & (dataset['Fare'] <= 14.454), 'Fare'] = 1
    dataset.loc[(dataset['Fare'] > 14.454) & (dataset['Fare'] <= 31), 'Fare']   = 2
    dataset.loc[ dataset['Fare'] > 31, 'Fare'] = 3
    dataset['Fare'] = dataset['Fare'].astype(int)

train = train.drop(['FareBand'], axis=1)
combine = [train, test]
    
#train.head(5)

train = train.drop(['AgeBand', 'Name', 'SibSp', 'Parch' ], axis = 1)
test = test.drop(['Name', 'SibSp', 'Parch'], axis = 1)
# splitting the dataset into x(independent variables) and y(dependent variables)

x_train = train.drop('Survived', axis = 1)
y_train = train.Survived

# print(x_train.shape)
# print(y_train.shape)

x_test = test

# print(x_test.shape)
horizontal_stack = pd.concat([x_train, y_train], axis=1)
horizontal_stack.index+=1
train_df=horizontal_stack
# train_df
x_test.index+=100000
test_df=x_test
#testcopy=test_df.copy()


In [ ]:
train_df

In [ ]:
test_df

# Building Model

In [ ]:
y=train_df['Survived']
X=train_df.drop('Survived', axis =1)
X_train,X_test,y_train,y_test = train_test_split(X,y,random_state=101, test_size=0.25)

In [ ]:
def build_classifier(optimizer):
    classifier = Sequential()
    classifier.add(Dense(units=10,kernel_initializer='uniform',activation='relu',input_dim=7))
    classifier.add(Dropout(rate = 0.2))
    classifier.add(Dense(units=64,kernel_initializer='uniform',activation='relu'))
    classifier.add(Dropout(rate = 0.2))
#     classifier.add(Dense(units = 8, kernel_initializer = 'uniform', activation = 'relu'))
#     classifier.add(Dropout(rate = 0.2))
    classifier.add(Dense(units=1,kernel_initializer='uniform',activation='sigmoid'))
    classifier.compile(optimizer=optimizer,loss='binary_crossentropy',metrics=['accuracy'])
    return classifier

In [ ]:
%%time
classifier = KerasClassifier(build_fn = build_classifier)
param_grid = dict(optimizer = ['Adam'],
                  epochs=[50,100,200],
                  batch_size=[16,32,64])
grid = HalvingRandomSearchCV(classifier, param_grid, scoring='accuracy',cv=10,n_jobs=-1)
grid_result = grid.fit(X_train, y_train)
best_parameters = grid.best_params_
best_accuracy = grid.best_score_

In [ ]:
print(grid.best_params_)
print(grid.best_score_)

In [ ]:
classifier = KerasClassifier(build_fn = build_classifier,
                             optimizer=best_parameters['optimizer'],
                             batch_size=best_parameters['batch_size'],
                             epochs=best_parameters['epochs'])

classifier.fit(X_train,y_train)
preds = classifier.predict(test_df)

**Submission File**

In [ ]:
# y_final = (preds > 0.5).astype(int).reshape(test_df.shape[0])

# output = pd.DataFrame({'PassengerId': test_df.index, 'Survived': y_final})
# output.to_csv('pred_ann.csv', index=False)

**Do upvote if you find this notebook helpful.**

This idea is still in intial phase so drop your views in comment section.